# Overview

In this notebook, we will go through implement the different decoding strategies by using the Huggingface Transformers library.


# Auto-regressive Language generation

It is based on the assumption that the probability distribution of a word sequence can be decomposed into the product of conditional next word distributions:

$$P(w_{1:T}|W_0)=\prod_{t=1}^{T}P(w_{t}|w_{1:t-1},W_{0}), w_{1:0}=\phi$$

and $W_{0}$ being the intial context word sequence. The length T of the word sequnce is usually determined on-the-fly and corresponds to the timestep t=T the EOS token is generated from $P(w_{t}|w_{1}:t-1, W_{0})$. Let's see the currently most prominent decoding methods:

* Greedy search
* Beam search
* Sampling

In [ ]:
pip install transformers==4.38.2

# Acknowledge

* https://huggingface.co/blog/how-to-generate
* https://huggingface.co/blog/ray-rag